In [1]:
from Modules.indexer import load_index
from Modules.cleaner import Cleaner
from Modules.query import Query_master


import pandas as pd
import re
from collections import defaultdict
from nltk.tokenize import TweetTokenizer
import numpy as np

In [2]:
INDEX_PATH = '../Data/index/index.json'
index = load_index(INDEX_PATH)

In [3]:
MUSICS_PARSED_PATH = '../Data/music_info/musics_parsed.csv'
musics_parsed = pd.read_csv(MUSICS_PARSED_PATH,index_col=0)

In [4]:
teste_2 = 'big OR small'

In [5]:
query_master = Query_master()

In [6]:
query = '(eu OR voce)(bonito OR bonita)'
result = query_master.parse_query(index,query)

In [7]:
def clean_querys_functions(query):
    patt = r'\(|\)|OR|NOT|NEAR'
    query = re.sub(patt,' ',query).strip()
    query = re.sub('  +',' ',query).strip()
    query = query_master.cleaner.clean_and_tokenize(query)
    return query

In [8]:
query_terms = clean_querys_functions(query)

In [9]:
term = 'eu'


In [10]:
def tf_idf(fij,ni,N):
    return np.log2(1+fij)*np.log2(N/ni)

In [11]:
def Ranking(result,musics_parsed):
    docs = set(result)
    rank_dict = {
        'Docs': [],
        'Ranks': []
    }

    for doc in docs:
        rank = 0
        for term in query_terms:
            term_dict = index[term]
            N = len(musics_parsed)
            ni = len(term_dict)

            if doc in term_dict.keys():
                fij = len(term_dict[doc])
            else:
                fij = 0

            rank += tf_idf(fij,ni,N)

        rank_dict['Docs'].append(doc)
        rank_dict['Ranks'].append(rank)
    
    out = pd.DataFrame(rank_dict).set_index('Docs')
    out.index = out.index.astype(int)
    return out

In [12]:
def search(query,index,musics_parsed):
    result = query_master.parse_query(index,query)
    Rank = Ranking(result,musics_parsed)
    Rank = Rank.join(musics_parsed).sort_values('Ranks',ascending=False)
    Rank.to_excel('resultado_da_pesquisa.xlsx')
    return Rank

In [16]:
query = 'olha que moça mais'
search(query,index,musics_parsed)

,Ranks,SName,SLink,Lyric,Idiom
Docs,,,,,
120943,16.079065,Ela Nem Tchum ||| Delley & Dorivan,/delley-dorivan/ela-nem-tchum.html,"Lá onde moro, um alguém que eu adoro. sua aten...",PORTUGUESE
63120,15.626609,Os Novos Donos ||| Buddy Oficial,/buddy-oficial/os-novos-donos.html,"Buddy. ""alô, alô, alô, teste"". Testa o mic da ...",PORTUGUESE
39194,15.358690,Baile Funky ||| Raimundos,/raimundos/baile-funky.html,Essa mulher tá me olhando. E me dizendo que me...,PORTUGUESE
62787,15.315602,Mansão Thug Stronda (part. Mr. Catra) ||| Bond...,/bonde-da-stronda/mansao-thug-stronda-part-mr-...,"Refrão(2x):. Mansão thug stronda formou, sem c...",PORTUGUESE
71730,15.190882,Quer Tomar Bomba? ||| Mag,/mag/quer-tomar-bomba.html,Quer tomar bomba?. Pode aplicar.. Mas eu não g...,PORTUGUESE
134383,14.797546,Rimas de Rodeio ||| Mococa & Paraíso,/mococa-paraiso/rimas-de-rodeio.html,Um peão apaixonado. Nunca perde o galeio. Olha...,PORTUGUESE
134265,14.797546,Rimas de Rodeio (Sem Locução) ||| Mococa & Par...,/mococa-paraiso/rimas-de-rodeio-sem-locucao.html,Um peão apaixonado. Nunca perde o galeio. Olha...,PORTUGUESE
66240,14.098003,Eu Tô Bem ||| Emicida,/emicida/eu-to-bem.html,"(Aí!) Irmão, olha a minha face,. Nunca vi prob...",PORTUGUESE
37288,13.087005,Teorema de Carlão ||| Pedra Letícia,/pedra-leticia/teorema-de-carlao.html,"Ei você ai!. Isso vai mudar sua vida então,. T...",PORTUGUESE


In [17]:
query_master.search(query,index,musics_parsed)['Lyric'].values[0]

'Lá no fundo sombrio de um bar cheio de mosca, camisa quadriculada por fora da calça larga e fosca. Barba melada de leite que escorria pela boca, e a baba de outros dias marcada na roupa.... Segurava meio trêmulo sua caneca de cabo cinza, caminhando lentamente com destino a saída. Começa mais um dia ou só mais um que se elimina, pelo homem com o nome Edvaldo Silva.. Não sabe o significado de afeto e amor, não conhece a pólen, as pétala rosa, só o espinho da flor. Lembrança da vida só na infância marcada com trauma e dor, ainda a memória do pai animal sem amor.... Que não tinha profissão e a um tempão fazia mixê, saía com homem ou mulher pra ter alguma coisa pra comer. Vendia farinha, maconha, seu corpo pra que quisesse ter, tudo dentro de casa pra família ver.... Foi crescendo Edvaldo Silva trancando num quarto com sua mãe Vilma, no berço atrás da cortina ouvia tudo que acontecia. Seu pai com outras meninas, sua mãe no meio também lhe servia, a noite inteira, música alta, droga e bebid